In [1]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
from tqdm import tqdm

In [2]:
class Dynamics:
    pass

In [3]:
hw4dynamics=Dynamics()
hw4dynamics.volcoeff = 3
hw4dynamics.alpha = -0.5
hw4dynamics.r = 0.05
hw4dynamics.S0 = 100



In [4]:
class Contract:
    pass

In [5]:
hw4contract=Contract()
hw4contract.T = 0.25
hw4contract.K = 100

In [6]:
class FD:
    pass

In [7]:
hw4FD=FD()
hw4FD.SMax=200
hw4FD.SMin=50
hw4FD.deltaS=0.1
hw4FD.deltat=0.0005


In [8]:
# You complete the coding of this function

def pricer_put_CEV_CrankNicolson(contract,dynamics,FD, BS = False):
# returns array of all initial spots,
# and the corresponding array of put prices

    volcoeff=dynamics.volcoeff
    alpha=dynamics.alpha
    r=dynamics.r
    T=contract.T
    K=contract.K

    # SMin and SMax denote the smallest and largest S in the _interior_.
    # The boundary conditions are imposed one step _beyond_, 
    # e.g. at S_lowboundary=SMin-deltaS, not at SMin.
    # To relate to lecture notation, S_lowboundary is S_{-J}
    # whereas SMin is S_{-J+1}

    SMax=FD.SMax
    SMin=FD.SMin
    deltaS=FD.deltaS
    deltat=FD.deltat
    N=round(T/deltat)
    if abs(N-T/deltat)>1e-12:
        raise ValueError('Bad time step')
    numS=round((SMax-SMin)/deltaS)+1
    if abs(numS-(SMax-SMin)/deltaS-1)>1e-12:
        raise ValueError('Bad time step')
    S=np.linspace(SMax,SMin,numS)    #The FIRST indices in this array are for HIGH levels of S
    S_lowboundary=SMin-deltaS

    putprice=np.maximum(K-S,0)

    ratio=deltat/deltaS
    ratio2=deltat/deltaS**2
    if BS is False:
        f = 0.5*(volcoeff**2)*(S**(2*(1+alpha)))
        g = 0
        h = -r*np.ones(len(S))
    else:
        f = 0.5*(volcoeff**2)*(S**2)
        g = 0
        h = -r*np.ones(len(S))
        
    F = 0.5*ratio2*f+0.25*ratio*g
    G = ratio2*f-0.50*deltat*h
    H = 0.5*ratio2*f-0.25*ratio*g
    
    RHSmatrix = diags([H[:-1], 1-G, F[1:]], [1,0,-1], shape=(numS,numS), format="csr")
    LHSmatrix = diags([-H[:-1], 1+G, -F[1:]], [1,0,-1], shape=(numS,numS), format="csr")
    # diags creates SPARSE matrices

    
    for t in tqdm(np.arange(N-1,-1,-1)*deltat):

        rhs = RHSmatrix * putprice
        
        #Now let's add the boundary condition vectors.
        #They are nonzero only in the last component:
        rhs[-1]=rhs[-1]+2*H[-1]*(K-S_lowboundary)

        putprice = spsolve(LHSmatrix, rhs)
        # numpy.linalg.solve, which expects arrays as inputs,
        # is fine for small matrix equations, and for matrix equations without special structure.
        # But for large matrix equations in which the matrix has special structure,
        # we want a more intelligent solver that can run faster 
        # by taking advantage of the special structure of the matrix.
        # Specifically, in this case, we want to use a solver that recognizes the SPARSE MATRIX structure.
        # Try spsolve, imported from scipy.sparse.linalg
        putprice = np.maximum(putprice, K-S)
        delta = (putprice[:-2]-putprice[2:])/(2*deltaS)
        gamma = (putprice[:-2] - 2*putprice[1:-1] + putprice[2:])/(deltaS**2)
    
    return(S, putprice, delta, gamma)
        

In [9]:
(S0_all, putprice, delta, gamma) = pricer_put_CEV_CrankNicolson(hw4contract,hw4dynamics,hw4FD)

100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1591.56it/s]


In [10]:
# pricer_put_CEV_CrankNicolson gives us option prices for ALL S0 from SMin to SMax
# But let's display only for a few S0 near 100:

displayStart = hw4dynamics.S0-hw4FD.deltaS*1.5 
displayEnd   = hw4dynamics.S0+hw4FD.deltaS*1.5
displayrows=np.logical_and(S0_all>displayStart, S0_all<displayEnd)
np.set_printoptions(precision=4, suppress=True)

In [11]:
print(np.stack((S0_all, putprice),1)[displayrows])

[[100.1      5.8704]
 [100.       5.9183]
 [ 99.9      5.9665]]


## c)

In [12]:
S0_idx = np.where(S0_all == 100)
delta_t0 = delta[S0_idx]
gamma_t0 = gamma[S0_idx]

print('Delta at S0 = 100, t=0 is equal to: {:.4f}'.format(delta_t0[0]))
print('Gamma at S0 = 100, t=0 is equal to: {:.4f}'.format(gamma_t0[0]))

Delta at S0 = 100, t=0 is equal to: -0.4833
Gamma at S0 = 100, t=0 is equal to: 0.0264


## d)

In [13]:
hw4dynamics.volcoeff = 0.3
(S0_all, putprice, delta, gamma) = pricer_put_CEV_CrankNicolson(hw4contract,hw4dynamics,hw4FD, BS = True)

100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1705.23it/s]


In [14]:
print(np.stack((S0_all, putprice),1)[displayrows])

[[100.1      5.8705]
 [100.       5.9169]
 [ 99.9      5.9636]]
